In [1]:
import pandas as pd
import numpy as np
from jinja2 import Template

In [2]:
def read_csv_file(file_path):
    """Reads a CSV file and returns a DataFrame."""
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    


In [4]:
df =read_csv_file('Berufsrollen_frueher_Archaeologen.csv')
df = df.replace({np.nan: None})
#berufsrolle = df.to_dict(orient='list')
#concept_scheme = list(df.iloc[0].items())

In [8]:
df.T

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
skos:prefLabel,Ägyptologe*in,Althistoriker*in,Anthropologe*in,Antiquar*in,Archäobotaniker*in,Archäologe*in,Archäozoologe*in,Architekt*in,Archivar*in,Bauarbeiter*in,...,Philologe*in,Politiker*in,Restaurator*in (archäologisch),Schriftsteller*in,Soldat*in,Staatsoberhaupt,Theolog*in,Unternehmer*in,Vermessungsingenieur*in,Grafiker*in
skos:altLabel,Ägyptenforscher*in,None,None,"Antikenhändler*in, Antiquitätenhändler*in",Paläobotaniker*in,None,Paläozoologe*in,Baumeister*in,Archivar*in,Bauhandwerker*in,...,Sprachforscher*in,Staatsfrau/Staatsmann,None,Autor*in,None,Herrscher*in,Religionswissenschaftler*in,Gewerbetreibende*r,Vermessungstechniker*in,Zeichner*in
dct:description,"Wissenschaftler*in, der/die das alte Ägypten e...","Historiker*in, der/die sich auf die Alte Gesch...","Wissenschaftler*in, der/die den Menschen und s...",Ein Antiquar/eine Antiquarin beschäftigt sich ...,"Wissenschaftler*in, die sich mit der Erforschu...","Wissenschaftler*in, die sich mit der Erforschu...","Wissenschaftler*in, die tierische Überreste au...","Person, die den Bau von Gebäuden und Anlagen p...",Verwaltet und erschließt historische Dokumente.,"Person, die auf einer Baustelle arbeitet.",...,"Wissenschaftler*in, die eine oder mehrere Spra...","Person, die in der Politik tätig ist.","Fachperson, die archäologische Funde konservie...","Person, die literarische Werke verfasst.","Person, die in den Streitkräften dient.",Herrscher*in eines Staates.,"Wissenschaftler*in, die sich mit religiösen Fr...","Person, die eine gewerbliche Tätigkeit ausübt.","Person, die mit entsprechender Ausbildung Verm...",Künstlerin oder Technikerin auf dem Gebiet der...
skos:definition,"Person, die sich mit der Erforschung der Gesch...","Wissenschaftler*in, die sich mit der politisch...","Person, die sich mit der biologischen und kult...","Spezialist*in für die Sammlung, die Pflege und...","Person, die pflanzliche Überreste aus archäolo...","Person, die durch Ausgrabungen und Analysen vo...","Person, die Knochen und andere tierische Überr...","Fachperson für die Planung, Gestaltung und Übe...","Person, die für die Erhaltung, Erschließung un...","Fachkraft, die handwerkliche Tätigkeiten auf B...",...,"Person, die sich mit der Analyse, Interpretati...","Vertreter*in des öffentlichen Lebens, die poli...","Person, die für die Erhaltung und Wiederherste...","Fachperson, die Romane, Erzählungen oder ander...",Angehörige*r der Streitkräfte.,Person an der Spitze der Exekutive eines Staates.,"Person, die sich mit der systematischen Erfors...","Person, die ein Unternehmen führt oder gegründ...","Fachperson, die Vermessungen durchführt und ge...","Person, die grafische Werke schafft oder techn..."
skos:scopeNote,Die Ägpytologie hat sich vor der Vor- und Früh...,Althistoriker*innen haben sich außer mit Texte...,Im Mittelpunkt stehen in diesem Kontext mensch...,Im Antikenhandel finden sich unter anderem leg...,None,Hier als Sammelbegriff für die verschiedenen D...,None,Bei Bauvorhaben und -arbeiten finden sich häuf...,None,Bei Bauvorhaben und -arbeiten finden sich häuf...,...,None,None,"Restaurator*in, die speziell im Bereich Archäo...",None,Bei Militäroperationen werden archäologische F...,Staatsoberhäupter gehörten zu den Sammlern arc...,None,None,Bei Vermessungsarbeiten wurden häufig archäolo...,Hier mit Bezug zu historischen oder archäologi...


In [ ]:
df = df.iloc[1:] #schmeißt die erste Zeile raus
df = df.drop(columns=["skos:Concept", "uri", "dct:title_de","skos:narrower", "skos:broader", "skos:related"])
df = df.T


In [ ]:
mapping= {}
i =0
for pref_label in df.iloc[0]:
    i +=1
    newindex =i
    #print (newindex)
    newname =pref_label.split("*")[0].lower()
    #print (pref_label, "->", newname)
    mapping[newindex] = newname

mapping
    

In [56]:
df1=df.rename(columns=mapping)

In [1]:
berufsrollen = df1.to_dict()
#berufsrollen

NameError: name 'df1' is not defined

In [58]:
with open('../arch_template.j2') as f:
    tmpl = Template(f.read())

archt_ttl =tmpl.render(trim_blocks=True, lstrip_blocks=True, concepts=berufsrollen, statics = [
    "skos:prefLabel",
    "dct:description",
    "skos:altLabel",
    "skos:definition"])
with open('arch_leiza.ttl', 'w') as f:
    f.write(archt_ttl)